## Crawling data from finnhub and yfinance and merge them to be a csv file

### 1) Finnhub (for stock news)

In [1]:
import finnhub
# get your api key from https://finnhub.io
finnhub_client =finnhub.Client(api_key="c5o758aad3i92b40uci0")

In [2]:
#load symbols from existing list
import pandas as pd
sp500 = pd.read_csv("../Technical_Assignment/csv/sp500.csv")
symbol = list(sp500.Symbol)
symbol.remove('BF.B')
symbol.remove('BRK.B')

In [3]:
for name in symbol[:1]:
    print(f'working on {name}')
    res = finnhub_client.company_news(name, _from="2021-02-15", to="2022-02-15")
    df = pd.DataFrame.from_dict(res)
df

working on A


,category,datetime,headline,id,image,related,source,summary,url
0,company,1644941281,Agilent Announces Cash Dividend of 21 Cents pe...,101327005,,A,Finnhub,"Agilent Technologies, Inc. today announced a q...",https://finnhub.io/api/news?id=8e5232300481463...
1,company,1644927660,"IPG Photonics (IPGP) Q4 Earnings Beat, Revenue...",101302522,https://www.nasdaq.com/sites/acquia.prod/files...,A,Nasdaq,IPG Photonics IPGP reported fourth-quarter 202...,https://finnhub.io/api/news?id=feb284cb14b9371...
2,company,1644831000,Why This 1 Computer and Technology Stock Could...,101016148,https://www.nasdaq.com/sites/acquia.prod/files...,A,Nasdaq,"Here at Zacks, we offer our members many diffe...",https://finnhub.io/api/news?id=be07071210b5e11...
3,company,1644598985,"Datadog (DDOG) Q4 Earnings Beat Estimates, Rev...",100437628,https://s.yimg.com/uu/api/res/1.2/Z_5XRRivEqCr...,A,Yahoo,Datadog's (DDOG) fourth-quarter 2021 results r...,https://finnhub.io/api/news?id=0026d12ece2d39c...
4,company,1644597120,Agilent Technologies Inc. stock underperforms ...,100545473,https://images.mktw.net/im-220105/social,A,MarketWatch,Shares of Agilent Technologies Inc. slid 3.15%...,https://finnhub.io/api/news?id=a846d36252d49ea...
...,...,...,...,...,...,...,...,...,...
182,company,1637074262,Workday (WDAY) to Report Q3 Earnings: What's i...,72558628,https://s.yimg.com/uu/api/res/1.2/vRg7a5yoCiFg...,A,Yahoo,Workday's (WDAY) fiscal third-quarter performa...,https://finnhub.io/api/news?id=793be026e86e8ec...
183,company,1637069401,Why Earnings Season Could Be Great for Agilent...,72579587,https://s.yimg.com/uu/api/res/1.2/OLLyh6ovH3rQ...,A,Yahoo,Agilent (A) is seeing favorable earnings estim...,https://finnhub.io/api/news?id=5f5adffaeee7d57...
184,company,1637052430,What to watch today: Stocks set to rise with s...,72557306,https://image.cnbcfm.com/api/v1/image/10697559...,A,CNBC,U.S. stock futures rose Tuesday as investors l...,https://finnhub.io/api/news?id=c4172bb5103d6c9...
185,company,1637007488,What's in Store for Applied Materials (AMAT) i...,72525435,https://s.yimg.com/uu/api/res/1.2/_e.wCX2kzGmD...,A,Yahoo,Applied Materials' (AMAT) fiscal fourth-quarte...,https://finnhub.io/api/news?id=45a536b24cae2ca...


In [4]:
from datetime import datetime
def ut2dt(timestamp):
    dt_object = datetime.fromtimestamp(timestamp)
    return dt_object.date()

In [5]:
df['Date']= df['datetime'].map(ut2dt)

### 2) yfinance (for stock price)

In [6]:
import yfinance as yf

In [7]:
end = datetime(2022,2,15)
end = datetime(end.year, end.month, end.day)
start = datetime(end.year - 1, end.month, end.day)

closing_df = yf.download(symbol[:1], start = start, end=end)['Adj Close']

[*********************100%***********************]  1 of 1 completed


In [8]:
dff = closing_df.to_frame()

In [9]:
dff.reset_index(inplace=True)

In [10]:
def re_date(bigdate):
    return bigdate.date()

In [11]:
dff['Date']=dff['Date'].map(re_date)

In [12]:
dff

,Date,Adj Close
0,2021-02-16,127.261612
1,2021-02-17,129.419937
2,2021-02-18,126.993065
3,2021-02-19,126.187431
4,2021-02-22,122.835548
...,...,...
248,2022-02-08,140.809998
249,2022-02-09,144.440002
250,2022-02-10,141.660004
251,2022-02-11,137.199997


In [13]:
dfnew = pd.merge(df, dff, how='inner', on='Date')

In [14]:
dfnew

,category,datetime,headline,id,image,related,source,summary,url,Date,Adj Close
0,company,1644831000,Why This 1 Computer and Technology Stock Could...,101016148,https://www.nasdaq.com/sites/acquia.prod/files...,A,Nasdaq,"Here at Zacks, we offer our members many diffe...",https://finnhub.io/api/news?id=be07071210b5e11...,2022-02-14,135.210007
1,company,1644598985,"Datadog (DDOG) Q4 Earnings Beat Estimates, Rev...",100437628,https://s.yimg.com/uu/api/res/1.2/Z_5XRRivEqCr...,A,Yahoo,Datadog's (DDOG) fourth-quarter 2021 results r...,https://finnhub.io/api/news?id=0026d12ece2d39c...,2022-02-11,137.199997
2,company,1644597120,Agilent Technologies Inc. stock underperforms ...,100545473,https://images.mktw.net/im-220105/social,A,MarketWatch,Shares of Agilent Technologies Inc. slid 3.15%...,https://finnhub.io/api/news?id=a846d36252d49ea...,2022-02-11,137.199997
3,company,1644564180,Surprising Analyst 12-Month Target For MGC,100323917,https://www.nasdaq.com/sites/acquia.prod/files...,A,Nasdaq,Looking at the underlying holdings of the ETFs...,https://finnhub.io/api/news?id=6c87e3911646d10...,2022-02-11,137.199997
4,company,1644510720,Agilent Technologies Inc. stock falls Thursday...,100545476,https://images.mktw.net/im-220105/social,A,MarketWatch,Shares of Agilent Technologies Inc. slipped 1....,https://finnhub.io/api/news?id=883a4af75f324dd...,2022-02-10,141.660004
...,...,...,...,...,...,...,...,...,...,...,...
158,company,1637074262,Workday (WDAY) to Report Q3 Earnings: What's i...,72558628,https://s.yimg.com/uu/api/res/1.2/vRg7a5yoCiFg...,A,Yahoo,Workday's (WDAY) fiscal third-quarter performa...,https://finnhub.io/api/news?id=793be026e86e8ec...,2021-11-16,161.177719
159,company,1637069401,Why Earnings Season Could Be Great for Agilent...,72579587,https://s.yimg.com/uu/api/res/1.2/OLLyh6ovH3rQ...,A,Yahoo,Agilent (A) is seeing favorable earnings estim...,https://finnhub.io/api/news?id=5f5adffaeee7d57...,2021-11-16,161.177719
160,company,1637052430,What to watch today: Stocks set to rise with s...,72557306,https://image.cnbcfm.com/api/v1/image/10697559...,A,CNBC,U.S. stock futures rose Tuesday as investors l...,https://finnhub.io/api/news?id=c4172bb5103d6c9...,2021-11-16,161.177719
161,company,1637007488,What's in Store for Applied Materials (AMAT) i...,72525435,https://s.yimg.com/uu/api/res/1.2/_e.wCX2kzGmD...,A,Yahoo,Applied Materials' (AMAT) fiscal fourth-quarte...,https://finnhub.io/api/news?id=45a536b24cae2ca...,2021-11-15,157.572464


In [15]:
cols = dfnew.columns.tolist()

In [16]:
cols

['category',
 'datetime',
 'headline',
 'id',
 'image',
 'related',
 'source',
 'summary',
 'url',
 'Date',
 'Adj Close']

In [17]:
cols.insert(0, cols.pop(cols.index('Date')))

In [18]:
cols

['Date',
 'category',
 'datetime',
 'headline',
 'id',
 'image',
 'related',
 'source',
 'summary',
 'url',
 'Adj Close']

In [19]:
dfnew = dfnew[cols]

In [20]:
dfnew

,Date,category,datetime,headline,id,image,related,source,summary,url,Adj Close
0,2022-02-14,company,1644831000,Why This 1 Computer and Technology Stock Could...,101016148,https://www.nasdaq.com/sites/acquia.prod/files...,A,Nasdaq,"Here at Zacks, we offer our members many diffe...",https://finnhub.io/api/news?id=be07071210b5e11...,135.210007
1,2022-02-11,company,1644598985,"Datadog (DDOG) Q4 Earnings Beat Estimates, Rev...",100437628,https://s.yimg.com/uu/api/res/1.2/Z_5XRRivEqCr...,A,Yahoo,Datadog's (DDOG) fourth-quarter 2021 results r...,https://finnhub.io/api/news?id=0026d12ece2d39c...,137.199997
2,2022-02-11,company,1644597120,Agilent Technologies Inc. stock underperforms ...,100545473,https://images.mktw.net/im-220105/social,A,MarketWatch,Shares of Agilent Technologies Inc. slid 3.15%...,https://finnhub.io/api/news?id=a846d36252d49ea...,137.199997
3,2022-02-11,company,1644564180,Surprising Analyst 12-Month Target For MGC,100323917,https://www.nasdaq.com/sites/acquia.prod/files...,A,Nasdaq,Looking at the underlying holdings of the ETFs...,https://finnhub.io/api/news?id=6c87e3911646d10...,137.199997
4,2022-02-10,company,1644510720,Agilent Technologies Inc. stock falls Thursday...,100545476,https://images.mktw.net/im-220105/social,A,MarketWatch,Shares of Agilent Technologies Inc. slipped 1....,https://finnhub.io/api/news?id=883a4af75f324dd...,141.660004
...,...,...,...,...,...,...,...,...,...,...,...
158,2021-11-16,company,1637074262,Workday (WDAY) to Report Q3 Earnings: What's i...,72558628,https://s.yimg.com/uu/api/res/1.2/vRg7a5yoCiFg...,A,Yahoo,Workday's (WDAY) fiscal third-quarter performa...,https://finnhub.io/api/news?id=793be026e86e8ec...,161.177719
159,2021-11-16,company,1637069401,Why Earnings Season Could Be Great for Agilent...,72579587,https://s.yimg.com/uu/api/res/1.2/OLLyh6ovH3rQ...,A,Yahoo,Agilent (A) is seeing favorable earnings estim...,https://finnhub.io/api/news?id=5f5adffaeee7d57...,161.177719
160,2021-11-16,company,1637052430,What to watch today: Stocks set to rise with s...,72557306,https://image.cnbcfm.com/api/v1/image/10697559...,A,CNBC,U.S. stock futures rose Tuesday as investors l...,https://finnhub.io/api/news?id=c4172bb5103d6c9...,161.177719
161,2021-11-15,company,1637007488,What's in Store for Applied Materials (AMAT) i...,72525435,https://s.yimg.com/uu/api/res/1.2/_e.wCX2kzGmD...,A,Yahoo,Applied Materials' (AMAT) fiscal fourth-quarte...,https://finnhub.io/api/news?id=45a536b24cae2ca...,157.572464


In [22]:
dfnew.to_csv('Sample_date.csv')